## Set up the run parameters parameters

In [ ]:
##use these parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
#subject_id_in = "100162_P"
#subject_id_out = "100162P"
#task_id_in = "VEP"
#task_id_in_et = "VEP" 
#task_id_out = "VEP"
#run_id = "1"
#session_id = "01"
#project_path = "/project/def-emayada/q1k/experimental/HSJ/"
#dataset_group = "experimental"
#site_code = "HSJ" #'MHC' or 'HSJ'
#et_sync = True
#html_figures = True
#read_tests = True

#use these empty parameters when executing this notebook from an automation script.
subject_id_in = ""
subject_id_out = ""
task_id_in = ""
task_id_in_et = "" 
task_id_out = ""
run_id = ""
session_id = ""
project_path = ""
dataset_group = ""
site_code = ""
et_sync = True
html_figures = False
read_tests = False

print('subject_id_in: ' + subject_id_in)
print('subject_id_out: ' + subject_id_out)
print('task_id_in: ' + task_id_in)
print('task_id_in_et: ' + task_id_in_et)
print('task_id_out: ' + task_id_out)
print('run_id: ' + run_id)
print('session_id: ' + session_id)
print('project_path: ' + project_path)
print('dataset_group: ' + dataset_group)
print('site_code: ' + site_code)


In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import re
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [ ]:
#define the DIN events associated with events of interest based on the task_id..
din_str, event_dict_offset = qit.set_din_str(task_id_out)
print("Initial DIN strings for " + task_id_out)
din_str

## Read the EEG file

In [ ]:
print("Input session identifier: " + subject_id_in)

In [ ]:
# generate the input paths
session_file_name_eeg, session_file_name_et = qit.generate_session_ids(dataset_group, project_path, site_code, task_id_in, subject_id_in, run_id)

In [ ]:
print("EEG session file name: " + session_file_name_eeg[0])

In [ ]:
if session_file_name_et and session_file_name_et[0]:
    print("ET session file name: " + session_file_name_et[0])
else:
    print("Could not find the session ET file.. abandoning ET sync portion of the initiation process.")
    et_sync = False
    

In [ ]:
# read the input EEG session
print('Reading: ' + session_file_name_eeg[0])
eeg_raw = mne.io.read_raw_egi(session_file_name_eeg[0])
eeg_raw_fresh=eeg_raw.copy() #make a fresh copy for later
device_info=eeg_raw.info['device_info']

In [ ]:
# peak... at the EEG channel types
channel_types = eeg_raw.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", eeg_raw.info['ch_names'])


In [ ]:
# show the sensor topography
fig = eeg_raw.plot_sensors(show_names=True)

## Get and modify the EEG event structures

In [ ]:
# create the EEG event structures
eeg_events = mne.find_events(eeg_raw, shortest_event = 1)
eeg_event_dict = qit.get_event_dict(eeg_raw, eeg_events, event_dict_offset)

In [ ]:
print('EEG event dict:')
eeg_event_dict

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original EEG event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)
# print the scatterplot to an html file for easy exploration.
if html_figures:
    fig.write_html("html_figures/eeg_event_times.html")

In [ ]:
if not din_str:
    print('Required EEG DIN events are missing... skipping EEG DIN check and DIN distance display')
else:
    #check that the din_str events exist in the eeg_event_dict..
    din_str = qit.din_check(eeg_event_dict, din_str)
    #get the distance between the DIN events of interest..
    din_diffs, din_diffs_time = qit.get_din_diff(eeg_events, eeg_event_dict, din_str)
    #build the figure...
    fig=px.scatter(x=din_diffs_time, y=din_diffs)
    fig.update_layout(title='Time between EEG DIN events of interest')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Inter event interval')
    # print the scatterplot to an html file for easy exploration.
    if html_figures:
        fig.write_html("html_figures/eeg_din_diffs.html")

In [ ]:
if not din_str:
    print('Required EEG DIN events are missing... skipping EEG stimulus onset DIN process')
else:
    #clean the events..
    eeg_events, eeg_event_dict = qit.eeg_clean_events(eeg_events, eeg_event_dict, din_str)
    # handle task specific EEG event interpretation..
    eeg_events, eeg_stims, eeg_iti, eeg_din_offset, eeg_event_dict, new_events = qit.eeg_task_events(eeg_events, eeg_event_dict, din_str, task_name=task_id_out)

In [ ]:
# print some stim event summaries
column_values = eeg_events[:, 2]

# Get unique values and their counts
unique_values, counts = np.unique(column_values, return_counts=True)

# Print the counts of each unique value
print("Counts of each event ID value:")
for value, count in zip(unique_values, counts):
    print(f"Event ID: {value}, Count: {count}")

# Print the event labels
print("Label\tValue")
for label, value in eeg_event_dict.items():
    print(f"{label}\t{value}")

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index... including new *_d DIN events if generated
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='DIN updated EEG event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
fig.show()
if html_figures:
    fig.write_html("html_figures/eeg_update_event_times.html")

In [ ]:
if not din_str:
    print('Required DIN events are missing... skipping stimulus DIN ITI display')
else:
    # peak... at the distance between *_d stim DIN events
    fig=px.scatter(x=eeg_stims[1:,0],y=eeg_iti)
    fig.update_layout(title='Stim DIN event Inter Trial Intervals (ITI)')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Stim DIN event ITI (ms)')
    fig.show()
    if html_figures:
        fig.write_html("html_figures/eeg_din_iti.html")

In [ ]:
if not din_str:
    print('Required DIN events are missing... skipping stimulus DIN event offset display')
else:
    # peak... at the distance between stim events and *_d stim DIN events
    fig=px.scatter(x=eeg_stims[:,0],y=eeg_din_offset)
    fig.update_layout(title='Stim DIN offsets')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Stim DIN offset (ms)')
    fig.show()
    if html_figures:
        fig.write_html("html_figures/eeg_stim_din_offset.html")

## Read the eye-tracking data

In [ ]:
et_raw, et_raw_df, et_events, et_event_dict = qit.et_read(session_file_name_et[0], blink_interp=False, fill_nans=False, resamp=False)

In [ ]:
if et_sync:
    # peak... at the ET channel types
    channel_types = et_raw.get_channel_types()
    print("ET Channel Types:", channel_types)
    print("ET Channel Names:", et_raw.info['ch_names'])
else:
    print("et_sync = False: not printing ET channle types")

In [ ]:
print(np.unique(et_raw.annotations.description))

In [ ]:
et_raw.plot(start=30, duration=1, scalings=dict(eyegaze=1e3,pupil=1e3))

## Handle the Eye-Tracking events

In [ ]:
if et_sync:
    print("ET event dict:", et_event_dict)
else:
    print("et_sync = False: not printing ET event dict")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=et_events[:,0],y=et_events[:,2])
    fig.update_layout(title='Original ET event times')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Event index')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    if html_figures:
        fig.write_html("html_figures/et_event_times.html")
else:
    print("et_sync = False: not plotting the original ET events")

In [ ]:
if et_sync:
    #do event cleaning..
    et_event_dict, et_events = qit.et_clean_events(et_event_dict, et_events)
    #do task specific event modifications..
    et_event_dict, et_events, et_raw_df = qit.et_task_events(et_raw_df,et_event_dict,et_events,task_id_out)
    print("updated ET event dict:", et_event_dict)
    # Extract the value for 'STIM_d' from the dictionary
    stim_d_value = et_event_dict['STIM_d']
    # Filter rows where the third column matches the 'STIM_d' value
    et_stims = et_events[et_events[:, 2] == stim_d_value]
    print('Number of stimulus onset DIN events: ' + str(len(et_stims)))
else:
    print("et_sync = False: not plotting the original ET events")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=et_events[:,0],y=et_events[:,2])
    fig.update_layout(title='Updated ET event times')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Event index')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    if html_figures:
        fig.write_html("html_figures/et_updated_event_times.html")
else:
    print("et_sync = False: not plotting the updated ET events")

## Examine the syncronization between the EEG and ET events and add [eeg,et]_synce_time events

In [ ]:
if et_sync:
    eeg_event_dict, et_event_dict, eeg_events, et_events, eeg_times, et_times = qit.eeg_et_align(
        eeg_event_dict, et_event_dict, 
        eeg_events, et_events, 
        eeg_stims, et_stims, 
        eeg_raw.info["sfreq"], et_raw.info["sfreq"])
else:
    print("et_sync = False: not checking eeg_times and et_times alignment")

In [ ]:
et_event_dict

In [ ]:
#et_events

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=eeg_times,y=et_times)
    fig.update_layout(title='EEG by ET stim times')
    fig.update_xaxes(title_text='EEG stim times')
    fig.update_yaxes(title_text='ET stim times')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    if html_figures:
        fig.write_html("html_figures/eeg_et_times.html")
else:
    print("et_sync = False: not plotting the EEG by ET event times")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    eeg_et_offset = eeg_times - et_times
    fig = px.scatter(y=eeg_et_offset)
    fig.update_layout(title='EEG ET stim event offset times')
    fig.update_xaxes(title_text='EEG ET stim times')
    fig.update_yaxes(title_text='EEG ET stim event offsets')
    fig.show()
    if html_figures:
        fig.write_html("html_figures/eeg_et_sync_offsets.html")
else:
    print("et_sync = False: not plotting the EEG by ET offset times")

## Write the raw structure to a BIDS directory in the project root.

In [ ]:
# Identify stim channels
stim_channels = [ch_name for ch_name, ch_type in zip(eeg_raw.info['ch_names'], eeg_raw.get_channel_types()) if ch_type == 'stim']
print(f"Stim channels to remove: {stim_channels}")

# Remove stim channels
eeg_raw.drop_channels(stim_channels)


In [ ]:
eeg_bids_path = qit.write_eeg(eeg_raw, 
              eeg_event_dict, 
              eeg_events, 
              subject_id_out, 
              session_id, 
              task_id_out, 
              project_path, 
              device_info)


In [ ]:
et_raw = qit.et_events_to_annot(et_raw, et_event_dict, et_events)
et_out_path = qit.write_et(et_raw, eeg_bids_path)

## read tests

In [ ]:
if read_tests:
    eeg_raw_in = mne_bids.read_raw_bids(bids_path=eeg_bids_path)
    eeg_raw_in.load_data()

In [ ]:
if read_tests:
    print(np.unique(eeg_raw_in.annotations.description))

In [ ]:
if read_tests:
    eeg_raw_in.plot(start=0, duration=60, scalings=dict(eeg=1e-3))

In [ ]:
if read_tests:
    et_raw_in = mne.io.read_raw_fif(et_out_path, preload=True)
    et_sfreq_in = et_raw_in.info['sfreq'] 
    et_events_in, et_event_dict_in  = mne.events_from_annotations(et_raw_in)

In [ ]:
if read_tests:
    print(np.unique(et_raw_in.annotations.description))

In [ ]:
if read_tests:
    et_raw_in.plot(start=0, duration=60, scalings=dict(eyegaze=1e2,pupil=1e3))

In [ ]:
if read_tests:
    et_raw_in.annotations.description

In [ ]:
if read_tests:
    et_raw_in.annotations.ch_names